<a href="https://colab.research.google.com/github/CopotronicRifat/Breast-Cancer-Detection-ML-Project/blob/main/INCEPTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Dropout, Flatten, Dense

def inception_module(input_layer, filters):
    tower1 = Conv2D(filters=filters[0], kernel_size=(1,1), padding='same', activation='relu')(input_layer)
    tower1 = Conv2D(filters=filters[1], kernel_size=(3,3), padding='same', activation='relu')(tower1)

    tower2 = Conv2D(filters=filters[2], kernel_size=(1,1), padding='same', activation='relu')(input_layer)
    tower2 = Conv2D(filters=filters[3], kernel_size=(5,5), padding='same', activation='relu')(tower2)

    tower3 = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same')(input_layer)
    tower3 = Conv2D(filters=filters[4], kernel_size=(1,1), padding='same', activation='relu')(tower3)

    output = concatenate([tower1, tower2, tower3], axis=3)
    return output

input_layer = Input(shape=(224, 224, 3))

x = Conv2D(filters=64, kernel_size=(7,7), strides=(2,2), padding='same', activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

x = Conv2D(filters=64, kernel_size=(1,1), padding='same', activation='relu')(x)
x = Conv2D(filters=192, kernel_size=(3,3), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

x = inception_module(x, filters=[64, 96, 128, 16, 32])
x = inception_module(x, filters=[128, 128, 192, 32, 96])
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

x = inception_module(x, filters=[192, 96, 208, 16, 48])
x = inception_module(x, filters=[160, 112, 224, 24, 64])
x = inception_module(x, filters=[128, 128, 256, 24, 64])
x = inception_module(x, filters=[112, 144, 288, 32, 64])
x = inception_module(x, filters=[256, 160, 320, 32, 128])
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

x = inception_module(x, filters=[256, 160, 320, 32, 128])
x = inception_module(x, filters=[384, 192, 384, 48, 128])

x = Flatten()(x)
x = Dropout(0.4)(x)
output_layer = Dense(units=1000, activation='softmax')(x)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
